In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import sklearn as skl

application_df= pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' & bring back the NAME column
application_df = application_df.drop(columns=['EIN'])
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [3]:
application_df_copy = application_df.copy()

In [4]:
# Look at NAME value counts for binning
name_counts = application_df['NAME'].value_counts()
# Determine which values to replace if counts are less than or equal to 5.
names_to_replace = list(name_counts[name_counts <= 5].index)

# Replace in dataframe
for app in names_to_replace:
    application_df_copy['NAME'] = application_df_copy['NAME'].replace(app,"Other")

# Check to make sure binning was successful
application_df_copy['NAME'].value_counts()

Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: NAME, Length: 355, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = application_df_copy['APPLICATION_TYPE'].value_counts()[application_df_copy['APPLICATION_TYPE'].value_counts() < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df_copy['APPLICATION_TYPE'] = application_df_copy['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_copy['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Changing the cutoff amount to reduce the number of classes.
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df_copy['CLASSIFICATION'].value_counts()[application_df_copy['CLASSIFICATION'].value_counts() < 1000].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df_copy['CLASSIFICATION'] = application_df_copy['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df_copy['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
is_object = application_df_copy.select_dtypes(include=['object']).columns
application_df_copy= pd.get_dummies(application_df_copy, columns = is_object)
application_df_copy.head()


,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,10531628,1,108590,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,10547893,1,5000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,10553066,1,6692,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,10556103,1,142590,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [8]:
application_df_copy.dtypes


EIN                         int64
STATUS                      int64
ASK_AMT                     int64
IS_SUCCESSFUL               int64
NAME_AACE INTERNATIONAL     uint8
                            ...  
INCOME_AMT_25000-99999      uint8
INCOME_AMT_50M+             uint8
INCOME_AMT_5M-10M           uint8
SPECIAL_CONSIDERATIONS_N    uint8
SPECIAL_CONSIDERATIONS_Y    uint8
Length: 400, dtype: object

In [9]:
# Split our preprocessed data into our features and target arrays
X = application_df_copy.drop(columns=["IS_SUCCESSFUL"])
y = application_df_copy["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [10]:
# Create a StandardScaler instances
scaler = skl.preprocessing.StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Number of features
num_features = X_train.shape[1]
print(f"Number of features: {num_features}")

Number of features: 399


In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=399))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu"))


# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                4000      
                                                                 
 dense_7 (Dense)             (None, 5)                 55        
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 4,061
Trainable params: 4,061
Non-trainable params: 0
_________________________________________________________________


In [17]:

# Compile model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Fit model
nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5217 - accuracy: 0.7533
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4448 - accuracy: 0.7900
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4364 - accuracy: 0.7919
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4322 - accuracy: 0.7926
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4305 - accuracy: 0.7934
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4285 - accuracy: 0.7942
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4277 - accuracy: 0.7950
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4268 - accuracy: 0.7946
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4262 - accuracy: 0.7949
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4253 - accura

In [19]:
# Evaluate the model using the training data
train_loss, train_accuracy = nn_model.evaluate(X_train_scaled, y_train, verbose=2)
print(f"Training Loss: {train_loss}, Training Accuracy: {train_accuracy}")


804/804 - 2s - loss: 0.4050 - accuracy: 0.8065 - 2s/epoch - 2ms/step
Training Loss: 0.40500879287719727, Training Accuracy: 0.8065230846405029


In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4509 - accuracy: 0.7859 - 341ms/epoch - 1ms/step
Loss: 0.45094773173332214, Accuracy: 0.785889208316803


In [21]:
# Try again with more epochs
# Fit model
nn_model.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.4090 - accuracy: 0.8044
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.4091 - accuracy: 0.8050
Epoch 3/200
804/804 [==============================] - 3s 3ms/step - loss: 0.4088 - accuracy: 0.8063
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.4085 - accuracy: 0.8050
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.4090 - accuracy: 0.8049
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.4083 - accuracy: 0.8056
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.4091 - accuracy: 0.8038
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.4086 - accuracy: 0.8049
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.4087 - accuracy: 0.8057
Epoch 10/200
804/804 [==============================] - 2s 3ms/step - loss: 0.4085 - accura

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4605 - accuracy: 0.7836 - 352ms/epoch - 1ms/step
Loss: 0.4604622721672058, Accuracy: 0.7835568785667419


In [24]:
# Changing the number of layers in the model

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model2 = tf.keras.models.Sequential()

# First hidden layer
nn_model2.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=399))

# Second hidden layer
nn_model2.add(tf.keras.layers.Dense(units=30, activation="sigmoid"))

# Third hidden layer
nn_model2.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))

# Output layer
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                4000      
                                                                 
 dense_7 (Dense)             (None, 5)                 55        
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 4,061
Trainable params: 4,061
Non-trainable params: 0
_________________________________________________________________


In [25]:

# Compile model
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:

# Fit model with the extra layer and more units
nn_model2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.4894 - accuracy: 0.7746
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4407 - accuracy: 0.7940
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4339 - accuracy: 0.7944
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4299 - accuracy: 0.7956
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4267 - accuracy: 0.7979
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4244 - accuracy: 0.7992
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4234 - accuracy: 0.7973
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4216 - accuracy: 0.7987
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4207 - accuracy: 0.7996
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4192 - accura

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4600 - accuracy: 0.7883 - 521ms/epoch - 2ms/step
Loss: 0.46000269055366516, Accuracy: 0.7883381843566895


In [ ]:
# Export our model to HDF5 file
# Save the model to an HDF5 file
import h5py

nn_model.save('AlphabetSoupCharity_Optimization.h5')